In [2]:
# Azure API Key
azure_api_key = 'be93af5916324304a1b7a0022dc4d673'
# Azure Endpoint
azure_endpoint = 'https://wbtz-openai-service.openai.azure.com/'
azure_deployment = 'wb_gpt4o'
api_version = '2024-02-15-preview'

azure_deployment = 'gpt-4.1'
api_version = '2025-01-01-preview'
import asyncio
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from autogen_agentchat.agents import AssistantAgent
model_client = AzureOpenAIChatCompletionClient(
    api_key=azure_api_key,
    azure_endpoint=azure_endpoint,
    model="gpt-4.1",
    azure_deployment=azure_deployment,
    api_version=api_version,
)
agent = AssistantAgent(name="assistant", model_client=model_client)

# result = await agent.run(task="阿瓦隆游戏的规则是什么？")
# print(result.messages[-1].content)



In [3]:
from autogen_agentchat.teams import Swarm
from autogen_agentchat.conditions import MaxMessageTermination
async def main():
    agent1 = AssistantAgent(
        "Alice",
        model_client=model_client,
        handoffs=["Bob"],
        system_message="You are Alice and you only answer questions about yourself.",
    )
    agent2 = AssistantAgent(
        "Bob", model_client=model_client, system_message="You are Bob and your birthday is on 1st January."
    )

    termination = MaxMessageTermination(3)
    team = Swarm([agent1, agent2], termination_condition=termination)

    stream = team.run_stream(task="What is bob's birthday?")
    async for message in stream:
        print(message)


await main()

id='86f9b6a0-8613-46ce-ab60-d3b7bd5ec1b2' source='user' models_usage=None metadata={} created_at=datetime.datetime(2026, 1, 28, 2, 14, 59, 801351, tzinfo=datetime.timezone.utc) content="What is bob's birthday?" type='TextMessage'
id='1640b66e-f69a-40ad-844d-c7930bcc2621' source='Alice' models_usage=RequestUsage(prompt_tokens=56, completion_tokens=35) metadata={} created_at=datetime.datetime(2026, 1, 28, 2, 15, 1, 713315, tzinfo=datetime.timezone.utc) content='I’m Alice, and I only answer questions about myself. I don’t provide information about Bob. If you have any questions about me, feel free to ask!' type='TextMessage'
id='f7abcfad-1079-48af-9c47-71f9a0b231d2' source='Alice' models_usage=RequestUsage(prompt_tokens=93, completion_tokens=13) metadata={} created_at=datetime.datetime(2026, 1, 28, 2, 15, 2, 270578, tzinfo=datetime.timezone.utc) content=[FunctionCall(id='call_x4G1b7iJuTNMKAG6HSUk38ml', arguments='{}', name='transfer_to_bob')] type='ToolCallRequestEvent'
id='6196b614-be8d-

In [4]:
awalon_rules = """
7人阿瓦隆游戏规则
核心定位: 7人阵营推理桌游, 分为好人方(4 人)与坏人方(3 人)，通过任务成败博弈 + 终局刺杀决定胜负，全程无玩家出局，侧重逻辑推理与身份伪装。
一、阵营与角色配置（固定）
好人方(蓝方)4 人
梅林：知晓除莫德雷德外的所有坏人身份，需隐藏自身以防被刺杀。
派西维尔：能看到梅林与莫甘娜，需自行分辨真假梅林，引导好人阵营。
忠臣x2: 无特殊技能, 仅需配合队友完成任务、掩护核心角色。
核心目标：完成 3 次任务，且终局保护梅林不被刺杀。
坏人方(红方)3 人
刺客：终局阶段负责刺杀梅林，可选择任务失败票。
莫甘娜：伪装梅林误导派西维尔，与队友互通身份。
奥伯伦：孤立坏人，无法与其他坏人互认，需自行辨识队友。
核心目标：搞砸 3 次任务直接胜利；或好人完成 3 次任务后，精准刺杀梅林逆转获胜。
阵营互通规则：坏人仅奥伯伦与队友互不见面，其余坏人开局互认；好人仅梅林、派西维尔有特殊信息，其余互不确认身份。
二、任务轮次与关键规则（核心）
1. 任务人数与失败判定（固定）
轮次	任务人数	失败票要求
1	2 人	1 张失败票 = 任务失败
2	3 人	1 张失败票 = 任务失败
3	3 人	1 张失败票 = 任务失败
4	4 人	2 张失败票 = 任务失败（特殊保护轮）
5	4 人	1 张失败票 = 任务失败
游戏率先达成 3 胜 / 3 负直接结束，无需进行全部 5 轮。
2. 单轮游戏流程
选队长：初始随机指定，之后按左手顺序轮流接任。
组队投票：队长按本轮任务人数组队（可含自己），全员投票，过半同意则组队成功；否则换队长重组。
强制规则：连续 4 次组队失败，第 5 任队长可强制组队执行任务。
执行任务：队员秘密出票，好人仅能出成功票，坏人可自由选择成功 / 失败票；队长洗牌后亮票，按失败票要求判定任务成败。
三、终局刺杀规则
触发条件：好人方率先完成 3 次任务。
操作流程：全员禁言，坏人方（不含奥伯伦）私下协商，指认 1 名玩家为梅林。
胜负判定：指认对象为梅林→坏人方胜利；指认为其他角色→好人方胜利。
"""

player_prompts = f"""
    你是一个7人阿瓦隆游戏的玩家。游戏规则如下：
    {awalon_rules}
    请根据上述规则，扮演一个玩家的角色，参与游戏并做出决策。
    所有的游戏玩家是：{{all_players}}。
    你的名字是：{{player_name}}。
    你的角色是：{{role}}。
    请根据你的角色和游戏进程，做出合理的决策和行动，你的目标是帮助你的阵营获胜。
    你的可能行动如下：
    1. 发言时，直接输出发言结果，不要输出内心想法，请注意不要暴露你的真实身份和信息, 可以适当误导其他玩家, 以保护你的阵营利益, 注意你的发言其他玩家都能看到。
    2. 投票队长提议的队伍时，你可以根据你的阵营利益，选择支持或反对队长提议的队伍, 你的投票其他玩家看不到。以<我的投票是: xxxx>的格式告诉主持人你的投票选择。发言与投票不同时进行
    3. 投票做任务时，你可以根据你的阵营利益，选择出成功票或失败票, 你的投票选择其他玩家看不到。以<我的投票是: xxxx>的格式告诉主持人你的投票选择。发言与投票不同时进行
    4. 当主持人把你分配为队长时，你需要选择合适的队员组队，并说服其他玩家投票通过你的队伍。
    5. 你只需要完成主持人交给你的任务，任务完成后转交给主持人。
    说明：你的发言偏向可能与投票和任务选择不一致，因为发言可以迷惑对手, 不能简单根据游戏玩家的发言来决定你的投票和任务选择。
    你的人设是：{{personality}}。

    根据你的角色-{{role}}, 你的角色能提前获取以下信息，请充分利用该信息进行游戏：
    {{info}}
    你的角色的注意事项是：{{role_notes}}
"""

# 主持人提示
host_prompt = f"""
    你是阿瓦隆游戏的主持人。游戏规则如下：
    {awalon_rules}
    你的任务是组织和引导游戏进程，确保所有玩家遵守规则。
    你需要根据游戏进程，向玩家提供必要的信息和提示。
    你需要在游戏过程中，适时总结和回顾游戏进展。
    你需要在游戏结束时，宣布胜利阵营并总结游戏亮点。
    初始随机选择队长, 不要老选player1作为初始队长。
    选定队长组队后，需要先由玩家发言后, 发言顺序为由队长开始，按顺时针方向依次发言，再进行投票表决
    不要把玩家的投票信息和执行任务的结果透露给其他玩家。玩家回复后，表示感谢就行，不用总结。
    投票结束时，需要总结有几票同意，几票反对, 几张失败票, 几张成功票, 并宣布任务结果。
    投票、发言、执行任务等环节需要你来组织和引导, 把任务交给对应的玩家即可，然后玩家把相应的结果交给你。
    玩家的发言与投票结果可能不一致, 不能简单根据发言来判断玩家的投票, 需要玩家明确完成投票动作。
    在发言时，请注意保持中立和客观，不偏袒任何一方, 也不要透露任何玩家的身份信息。
    游戏结束时, 输出terminate。

    游戏信息如下：
    {{game_info}}

"""

players = ["player1", "player2", "player3", "player4", "player5", "player6", "player7"]
roles = {
    "player5": "梅林",
    "player1": "派西维尔",
    "player7": "忠臣",
    "player4": "忠臣",
    "player2": "刺客",
    "player6": "莫甘娜",
    "player3": "奥伯伦"
}
players_info = {
    "player5": {"role": "梅林", "personality": "聪明且谨慎, 发言时注意迷惑对方阵营, 不要暴露出你是梅林", "info": "知道除莫德雷德外的所有坏人身份, 即player2、player6和player3是坏人", "role_notes": "隐藏身份，避免被刺杀"},
    "player2": {"role": "刺客", "personality": "狡猾且有野心", "info": "知道player6是坏人", "role_notes": "在游戏结束时刺杀梅林以帮助坏人阵营获胜"},
    "player3": {"role": "奥伯伦", "personality": "孤独且神秘", "info": "无", "role_notes": "孤立坏人，无法与其他坏人互认，需自行辨识队友"},
    "player4": {"role": "忠臣", "personality": "忠诚且勇敢", "info": "无"},
    "player1": {"role": "派西维尔", "personality": "机智且善于观察, 注意保护梅林", "info": "知道player5和player6是梅林和莫甘娜, 但是不知道谁是谁", "role_notes": "识别真假梅林，保护真正的梅林"},
    "player6": {"role": "莫甘娜", "personality": "狡诈且富有魅力", "info": "知道player2是坏人", "role_notes": "伪装成梅林，误导派西维尔"},
    "player7": {"role": "忠臣", "personality": "忠诚且勇敢", "info": "无"}
}

In [5]:
from autogen_agentchat.agents import AssistantAgent
from autogen_core.model_context import UnboundedChatCompletionContext
from autogen_core.models import AssistantMessage, LLMMessage, ModelFamily, UserMessage
from typing import List
from autogen_agentchat.messages import HandoffMessage

class AwalonAssistantAgent(AssistantAgent):

    @staticmethod
    async def _add_messages_to_context(
        model_context,
        messages,
    ) -> None:
        """
        Add incoming messages to the model context.
        """
        for msg in messages:
            if isinstance(msg, HandoffMessage):
                for llm_msg in msg.context:
                    await model_context.add_message(llm_msg)
            await model_context.add_message(msg.to_model_message())

# 过滤掉投票信息的模型上下文
class FilterVoteInfoContext(UnboundedChatCompletionContext):
    "A model context for filtering out vote information from AssistantMessage."

    async def get_messages(self) -> List[LLMMessage]:
        messages = await super().get_messages()
        # Filter out thought field from AssistantMessage.
        messages_out: List[LLMMessage] = []
        for message_item in messages:
            message = message_item.copy()
            if isinstance(message, UserMessage):
                # 替换角色名为***
                roles_to_hide = ["梅林", "派西维尔", "忠臣", "刺客", "莫甘娜", "奥伯伦"]
                for role in roles_to_hide:
                    if message.content and isinstance(message.content, str):
                        message.content = message.content.replace(role, "***")
                    # if message.thought and isinstance(message.thought, str):
                    #     message.thought = message.thought.replace(role, "***")
            if isinstance(message, UserMessage) and message.source != "Host":
                # if message.thought and "我的投票是" in message.thought:
                #     message.thought = "[内容已隐藏]"
                if message.content and "我的投票是" in message.content:
                    message.content = "[内容已隐藏]"
            # print("Filtered message:", message)
            messages_out.append(message)
        return messages_out

In [8]:
from autogen_agentchat.teams import Swarm
from autogen_agentchat.conditions import MaxMessageTermination
from autogen_agentchat.conditions import TextMentionTermination
from autogen_core.model_context import BufferedChatCompletionContext
async def main():
    player_agents = []
    for player in players_info:
        prompt = player_prompts.format(
            role=players_info[player]["role"],
            all_players=players,
            player_name=player,
            personality=players_info[player]["personality"],
            info=players_info[player]["info"],
            role_notes=players_info[player].get("role_notes", "无")
        )
        # print(f'Creating agent for {player} with role {players_info[player]["role"]}, prompt:\n{prompt}\n')
        player_context = FilterVoteInfoContext()
        agent = AwalonAssistantAgent(
            name=player,
            handoffs=["Host"],
            model_client=model_client,
            model_context=player_context,
            system_message=prompt
        )
        player_agents.append(agent)
    host_agent = AwalonAssistantAgent(
        name="Host",
        handoffs=players,
        model_client=model_client,
        system_message=host_prompt.format(game_info=roles)
    )
    max_msg_termination = MaxMessageTermination(20)
    text_mention_termination = TextMentionTermination("terminate")
    team = Swarm([host_agent] + player_agents, termination_condition=text_mention_termination,)

    try: 
        f = open('avalon_game_log.txt', 'w', encoding='utf-8')
        f.write('=== 阿瓦隆游戏日志 ===\n')
        stream = team.run_stream(task="游戏开始")
        async for message in stream:
            try:
                f.write(f'role: {message.source}, content: {message.content}\n')
                if 'ransfer' in message.content or (type(message.content) is not str):
                    continue
                print(f'role: {message.source}, content: {message.content}\n')
            except Exception as e:
                print(f'Error printing message from {message}: {e}')
    except Exception as e:
        print(f'Error during team interaction: {e}')

    for agent in player_agents + [host_agent]:
        print(f'=== Conversation model context for {agent.name} ===')
        # 尝试获取 agent 内部状态
        if hasattr(agent, '_model_context'):
            print(await agent._model_context.get_messages())
        print('=========================================')


await main()

role: user, content: 游戏开始

role: Host, content: 游戏开始，欢迎大家来到阿瓦隆桌游！本局为7人标准配置，现在为大家介绍阵营和角色分布：

- 好人阵营（蓝方）：4人（梅林、派西维尔、忠臣x2）
- 坏人阵营（红方）：3人（刺客、莫甘娜、奥伯伦）

每位玩家的具体身份已分配，只有各自清楚自身身份。游戏将依照规则推进，不会透露任何玩家具体身份信息。

现在，随机选择初始队长。本局的第一任队长为：player3。

第一轮任务需要2名玩家参与。请player3决定本轮任务的2人组队名单。选择完成后，按顺时针方向（player3 → player4 → player5 → player6 → player7 → player1 → player2）依次进行发言。

player3，请宣布您的组队名单。



/var/folders/7m/l3ny_hkx7wg3srx7mw906k3r0000gn/T/ipykernel_81094/559868124.py:32: PydanticDeprecatedSince20: The `copy` method is deprecated; use `model_copy` instead. See the docstring of `BaseModel.copy` for details about how to handle `include` and `exclude`. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  message = message_item.copy()


role: player3, content: 我宣布，本轮任务组队名单为：player3 和 player4。

大家请发表你的看法。

role: player4, content: 这个组队搭配中，有一人是自己队长，有一人是随机选的。如果没有更多的信息，只能说暂时没有问题，但希望大家关注后续任务失败情况，及时调整怀疑对象。

role: player5, content: 目前第一轮信息有限，不过两人队伍本来容错率低，可以先观察表现。player3选择player4，有一定概率是尝试自保，也可能是随机选取。如果团队有更优建议可以补充，否则我暂时不反对这一轮。

（请下一位玩家继续发言。）

role: player6, content: 这个队伍没有太明显的问题。第一轮通常很难看出端倪，建议大家都关注任务结果，留意谁在发言时表现得过于积极或过于谨慎。希望大家理性分析，不要轻易下定论。

（请下一位玩家继续发言。）

role: player7, content: 目前是第一轮任务，信息确实不多。我认为player3和player4这样的组合比较常见，前期我们可以通过任务结果积累线索，大家理性分析，不要急于怀疑。建议大家把重点放在后续几轮的投票和行为逻辑上。

请下一位玩家继续发言。

role: player1, content: player3和player4的组合属于常规搭配，目前没有太大问题。建议大家本轮关注任务票数，如果出现失败，可以据此分析嫌疑人。希望大家理性投票，后续多留心发言内容。

role: player2, content: 我觉得player3和player4的组合比较平衡，第一轮本就没有太多信息。如果本轮任务顺利通过，可以对他们暂时存疑；如果失败，其中必然有坏人。建议大家先观察为主，本轮我没有异议。

（主持人可以进入投票环节。）

role: Host, content: 所有玩家已依次发言，现进入组队投票环节。

请player1至player7依次进行“是否同意本轮player3和player4参与任务”的投票。请各位玩家回复自己的投票决定（同意/反对）即可。

player1，请率先投票。

role: player1, content: <我的投票是: 同意>

role: player2, content: <我的投票是: 

/Users/zhaoweisong/anaconda3/envs/langchain/lib/python3.10/site-packages/autogen_agentchat/agents/_assistant_agent.py:1245: UserWarning: Multiple handoffs detected. Only the first is executed: ['transfer_to_player3', 'transfer_to_player4']. Disable parallel tool calls in the model client to avoid this warning.
  handoff_output = cls._check_and_handle_handoff(


role: player3, content: <我的投票是: 失败>

role: Host, content: 收到player3的任务票。

请player4提交本轮任务票（成功/失败）。

role: player4, content: <我的投票是: 成功>

role: Host, content: 本轮任务票已收齐：

- 失败票：1张
- 成功票：1张

根据规则，第一轮只需1张失败票即判定任务失败。因此：

本轮任务失败，坏人阵营获得一分。

目前任务进度：
- 任务成功：0次
- 任务失败：1次

接下来将由player4成为下一任队长。player4，请选择本轮任务3人组队名单。

role: player4, content: 本轮我提名的任务队伍是：player4、player1、player6。

大家请发表意见，希望通过分散选择，尽量排查可疑对象，收集更多信息。

role: player5, content: player4选择了自己、player1和player6进入第二轮任务。上一轮任务失败，说明队里必有一名坏人。这次队伍只保留了player4，还加入了两位新成员，player6是上一轮发言谨慎的玩家，值得关注。

目前来说，这个组合比前一轮多出新面孔，有助于筛查嫌疑。如果大家没有比这个更稳妥的想法，这一轮我倾向于观察，但也建议大家理性分析票数和结果，及时调整怀疑对象。

（请下一位玩家发言。）

role: player6, content: player4这轮选择了不同搭配，说明有排查嫌疑的意图。player1是新加入的成员，player4刚刚在失败任务队伍里，我自己也在队里，三人组合可以产生新的信息。

我觉得这组队不算特别保险，但如果大家有更好的建议也可以提出。建议这一轮关注票型与任务结果，结合上一轮有利于后期推理。

role: player7, content: player4这轮更换了部分队员，说明有尝试查验的思路。我们可以通过这轮任务的结果进一步分析每个人的嫌疑。建议大家结合上一轮的失败情况，观察这次新加入的队员表现，相信通过推理可以逐步明确线索。

请下一位玩家继续发言。

role: player1, content: player4更换组队名单，给这轮带来新的信息。player1和player6作为新成员，可

...

role: player4, content: <我的投票是: 成功>

role: Host, content: 收到player4的任务票。

请player1提交本轮任务票（成功/失败）。

role: player1, content: <我的投票是: 成功>

role: Host, content: 收到player1的任务票。

请player6提交本轮任务票（成功/失败）。

role: player6, content: <我的投票是: 失败>

role: Host, content: 本轮任务票已收齐：

- 失败票：1张
- 成功票：2张

根据规则，第二轮只需1张失败票即判定任务失败。因此：

本轮任务失败，坏人阵营再得一分。

目前任务进度：
- 任务成功：0次
- 任务失败：2次

下一轮队长为player5。player5，请选择本轮任务3人组队名单。

role: player5, content: 本轮我们连续两轮任务失败，说明坏人分布较广，上一轮组进player4、player1、player6队伍也被黑票带走。这对于好人来说压力较大，需要尽快拉回优势。

我提名的本轮队伍是：player5、player7、player1。

我的理由是：player7至今未进过任务，表现相对中立且发言理性，player1刚刚参与，上轮没有明显的违规点，我自己则代表透明负责，希望给团队带来参考价值。也建议大家对这一轮结果高度关注，及时推翻怀疑方向。

请大家发表意见，如果有更合适组合，可以及时提出。

role: player6, content: player5选择了自己、player7和player1进入任务。从目前局势来看，连输两轮任务需要我们慎重选择。这次player7是首次被选入队伍，player1已连续两轮被带队，不排除被试探或掩护的可能。建议大家重点关注本轮任务结果，同时结合之前失败的两轮进行横向分析。当前局势对好人确实不利，希望通过这轮能找到更多线索。

role: player7, content: player5本轮选了自己、player7和player1。这是player7（我）首次进入任务队伍，player1连续被提名，player5自荐显得比较积极。如果大家暂时没有更好的选择，可以考虑先通过这一组，重点分析本轮任务结果

/Users/zhaoweisong/anaconda3/envs/langchain/lib/python3.10/site-packages/autogen_agentchat/agents/_assistant_agent.py:1245: UserWarning: Multiple handoffs detected. Only the first is executed: ['transfer_to_player5', 'transfer_to_player1', 'transfer_to_player7']. Disable parallel tool calls in the model client to avoid this warning.
  handoff_output = cls._check_and_handle_handoff(


role: player5, content: <我的投票是: 成功>

role: Host, content: 收到player5的任务票。

请player7提交本轮任务票（成功/失败）。

role: player7, content: <我的投票是: 成功>

role: Host, content: 收到player7的任务票。

请player1提交本轮任务票（成功/失败）。

role: player1, content: <我的投票是: 成功>

role: Host, content: 本轮任务票已收齐：

- 失败票：0张
- 成功票：3张

本轮任务成功，好人阵营获得一分！

目前任务进度：
- 任务成功：1次
- 任务失败：2次

下一轮进入第四轮任务，需要4名成员参与。新任队长为player6。player6，请选择本轮4人组队名单。

role: player6, content: 本轮队伍我提名：player6（我自己）、player2、player3、player7

我选择这个组合的理由是，player7刚刚完成了一次成功任务，有一定参考价值，player2和player3之前并未频繁同队，可以试探新组合；我自己也愿意承担责任。如果大家有其他人选建议，也欢迎补充讨论，关键在于获得更多的信息帮助完成任务。

希望大家理性分析，集中精力保证这一轮任务顺利，不能再出现失败。

role: player7, content: player6这次选择了player2、player3和player7（我）组成四人队伍，整体来看有新老成员混合，能试探出更多线索。player7刚刚所在任务组成功，player2和player3之前配合较少，便于交叉验证。

建议大家多角度分析组队合理性，关注队员之间的互动和后续任务表现，用实际行动找出坏人，争取再拿一分！

请下一位玩家继续发言。

role: player1, content: player6这次选择的队伍组合有新思路，既有之前成功任务的成员player7，也有相对未深度配合过的player2和player3。队伍搭配可以为大家提供新的观察信息。建议本轮务必关注任务结果和投票倾向，结合前几轮数据，理性识别嫌疑对象，逐步缩小范围。

role: player2, content: player6这

/Users/zhaoweisong/anaconda3/envs/langchain/lib/python3.10/site-packages/autogen_agentchat/agents/_assistant_agent.py:1245: UserWarning: Multiple handoffs detected. Only the first is executed: ['transfer_to_player6', 'transfer_to_player2', 'transfer_to_player3', 'transfer_to_player7']. Disable parallel tool calls in the model client to avoid this warning.
  handoff_output = cls._check_and_handle_handoff(


role: player6, content: <我的投票是: 失败>

role: Host, content: 收到player6的任务票。

请player2提交本轮任务票（成功/失败）。

role: player2, content: <我的投票是: 失败>

role: Host, content: 收到player2的任务票。

请player3提交本轮任务票（成功/失败）。

role: player3, content: <我的投票是: 失败>

role: Host, content: 收到player3的任务票。

请player7提交本轮任务票（成功/失败）。

role: player7, content: <我的投票是: 成功>

role: Host, content: 本轮任务票已收齐：

- 失败票：3张
- 成功票：1张

第四轮任务需要2张失败票判定失败，因此：

本轮任务失败，坏人阵营率先达成三次任务失败，获得本局胜利！

游戏总结：
坏人阵营凭借关键轮次团结一致，连续破坏三次任务，最终取得胜利。好人阵营在前三轮尝试多样组合，但未能有效排查关键身份。尤其是本轮，坏人队伍纷纷投出失败票，直接终结比赛。值得关注的是，player5带队的第三轮成功为好人带来希望，但后续未能延续胜势。

感谢大家参与本局阿瓦隆！欢迎下次再战！

terminate

Error printing message from messages=[TextMessage(id='99d522a4-ca02-4e92-b568-5448d6c2f62b', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2026, 1, 28, 3, 6, 1, 229352, tzinfo=datetime.timezone.utc), content='游戏开始', type='TextMessage'), TextMessage(id='9eba9040-46e0-44a7-8a35-856fd4634c54', source='Host', models_usage=RequestUsage(prompt_tokens=1242, completion_token